In [ ]:
from dotenv import load_dotenv
import os
import cloudinary
import cloudinary.uploader
import cloudinary.api
import time
import requests


load_dotenv()

cloudinary.config(
    cloud_name=os.getenv("CLOUD_NAME"),
    api_key=os.getenv("CLOUD_API_KEY"),
    api_secret=os.getenv("CLOUD_API_SECRET")
)

files_to_upload = [
    "ecg_model_mlp.pth",
    "minmaxscaler.pkl",
    "predicted_data.csv"  
]

folder_local = "./"
os.makedirs(folder_local, exist_ok=True)

cloud_folder = "githubRepo-Ecg-Proyecto"

try:
    cloudinary.api.delete_resources_by_prefix(cloud_folder)
    print(f"DELETING ALL FILES in {cloud_folder}...")
except Exception as e:
    print(f"CAN'T CLEAN UP '{cloud_folder}': {e}")

uploaded_files = []

for file_name in files_to_upload:
    file_path = os.path.join(folder_local, file_name)
    if os.path.exists(file_path):
        result = cloudinary.uploader.upload(
            file_path,
            resource_type="raw",
            folder=cloud_folder,
            use_filename=True,
            unique_filename=False
        )
        uploaded_files.append(file_name)
        print(f"[OK] Subido: {file_name}")
    else:
        print(f"[SKIP] Archivo no encontrado localmente: {file_path}")

try:
    payload = {
        "status": "completed",
        "uploaded": uploaded_files
    }
    r = requests.post("http://localhost:3333/", json=payload, timeout=10)
    print(f"[CONFIRMATION] Enviada al backend: {r.status_code}, {r.text}")

    time.sleep(10)

    payload_off = {
        "status": "inactive",
        "uploaded": uploaded_files
    }
    r2 = requests.post("http://localhost:3333/", json=payload_off, timeout=10)
    print(f"[CONFIRMATION OFF] Enviada al backend: {r2.status_code}, {r2.text}")

except Exception as e:
    print(f"[ERROR] No se pudo enviar la confirmación al backend: {e}")